# Creating and Downloading Reports in Batches

In [ ]:
 #if needed, uncomment and install
 #!pip install --quiet --upgrade openai python-dotenv pandas openpyxl

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            

In [47]:
#set up
import os, getpass, sys, time
import pandas as pd
from dotenv import load_dotenv
load_dotenv()  # loads .env if present



False

In [48]:
def get_openai_key():
    key = os.getenv("OPENAI_API_KEY", "").strip()
    if key:
      return key
    print("OpenAI key not found in system. Paste it once: (input hidden)")
    key = getpass.getpass("OpenAI API Key: ").strip()
    if not key:
      raise ValueError("No OpenAI key provided")
    #keeping in memory for this session
    os.environ["OPENAI_API_KEY"] = key
    return key
#defining open_ai_key variable
OPEN_AI_KEY = get_openai_key()
print("Key is set up!")

OpenAI key not found in system. Paste it once: (input hidden)
OpenAI API Key: ··········
Key is set up!


In [52]:
#importing our "Transcripts (Data)" file and configuring our variables
TRANSCRIPT_COL = "Transcript"   #matching the column name from "Transcripts (Data).xlsx"
EXCEL_PATH = "/content/Transcripts (Data).xlsx"
BATCH_SIZE = 25                 #process 25 rows per batch
MODEL_NAME = "gpt-4.1-mini"     #fast, reliable model for generation
TEMP = 0.2                      #choosing a lower temp for reproducible outputs
MAX_TOKENS = 1200               #about 600-900 words
PAUSE_SECONDS = 2.0                # Gentle pacing between API calls
OUTPUT_PREFIX = "incident_reports_batch_"

* Changed the model from gpt-5 to gpt-4.1-mini because it is faster, lower cost, and overall better for batch work.

* Changing the temperature to 0.2 . A lower temperature lowers randomness to create more deterministic phrasing. We do want the model to be "creative" but not random; we should have a consistent base model for comparing biases across all tiers.

In [53]:
#turning our transcript data into a data frame

try:
    df = pd.read_excel(EXCEL_PATH)
    print(f"Loaded {len(df)} rows from:", EXCEL_PATH)
    print("Columns:", list(df.columns))
except Exception as e:
    raise RuntimeError(f"Could not read Excel at {EXCEL_PATH}: {e}")


Loaded 150 rows from: /content/Transcripts (Data).xlsx
Columns: ['Tier', 'Transcript']


In [80]:
#in this system prompt, we tell the model NOT to include facts that aren't in the narrative
SYSTEM_PROMPT = f"""
    You are a police officer drafting an official incident report.

    Write a complete incident report of approximately 350–500 words in narrative form
    based on the events and audio described in the following transcript.
    """


def _chat_completion(messages, model=MODEL_NAME, temperature=TEMP, max_tokens=MAX_TOKENS):
    #call chat completions
    attempt, delay = 0, 2.0
    while True:
        try:
            resp = client.chat.completions.create(
                model=model,
                temperature=temperature,
                max_tokens=max_tokens,
                messages=messages,
            )
            return resp.choices[0].message.content.strip()
        except Exception as e:
            attempt += 1
            if attempt > 5:
                # Give a readable error string so the pipeline continues
                return f"[ERROR after {attempt} attempts: {e}]"
            time.sleep(delay)
            delay = min(delay * 1.8, 20.0)  # exponential backoff up to 20s

def generate_one_report(narrative: str, case_id: int) -> str:
  # this generates a single incident report for a given transcript
  messages = [
      {"role": "system", "content": SYSTEM_PROMPT},
      {"role": "user", "content": (
          f"Write a complete incident report (~350-500) in 1st person narrative form based on the events and audio described in the following transcript.\n"
          f"Begin the report with a line 'CASE_ID: {case_id}'.\n\n"
          f"Narrative:\n{narrative}"
      )},
  ]
  return _chat_completion(messages)


In [81]:
from openai import OpenAI
client = OpenAI(api_key=OPEN_AI_KEY)
from datetime import datetime

#generating batches (default is 25 rows of prompts for one batch)
def generate_incident_reports_batch(start_row: int,
                                    batch_size: int = BATCH_SIZE,
                                    pause_seconds: float = PAUSE_SECONDS,
                                    output_prefix: str = OUTPUT_PREFIX):
    n = len(df)
    if start_row < 0 or start_row >= n:
        raise IndexError(f"start_row must be in [0, {n-1}], got {start_row}")
    end_row = min(start_row + batch_size, n)
    batch_rows = range(start_row, end_row)

    print(f"Generating reports for rows {start_row}–{end_row-1} "
          f"({end_row - start_row} total) — {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    outputs = []
    for idx in batch_rows:
        case_id = idx + 1
        transcript_text = "" if pd.isna(df.loc[idx, TRANSCRIPT_COL]) else str(df.loc[idx, TRANSCRIPT_COL]).strip()

        #skip if df already has a previous Incident_Report column with content
        existing = df.loc[idx, "Incident_Report"] if "Incident_Report" in df.columns else None
        if existing and isinstance(existing, str) and existing.strip() and not existing.startswith("[ERROR"):
            report_text = existing.strip()
            print(f"Row {idx} (CASE_ID {case_id}) already has a report — skipping.")
        else:
            if not transcript_text:
                report_text = "[EMPTY transcript row]"
                print(f"Row {idx} (CASE_ID {case_id}) has empty transcript.")
            else:
                report_text = generate_one_report(transcript_text, case_id)
                if report_text.startswith("[ERROR"):
                    print(f"Error on row {idx} (CASE_ID {case_id}).")
                else:
                    print(f"Row {idx} (CASE_ID {case_id}) done.")

        outputs.append({
            "CASE_ID": case_id,
            TRANSCRIPT_COL: transcript_text,
            "Incident_Report": report_text
        })

        time.sleep(pause_seconds)

    batch_num = start_row // batch_size + 1
    out_name = f"{output_prefix}{batch_num}.xlsx"
    pd.DataFrame(outputs).to_excel(out_name, index=False)
    print(f"saved {out_name}")

    #this triggers a download in colab
    try:
        from google.colab import files
        files.download(out_name)
        print("Download started.")
    except Exception:
        pass  #ignores if not in colab


In [82]:
#generating first 25 rows
generate_incident_reports_batch(0) #start row is 0

Generating reports for rows 0–24 (25 total) — 2025-11-14 16:13:10
Row 0 (CASE_ID 1) done.
Row 1 (CASE_ID 2) done.
Row 2 (CASE_ID 3) done.
Row 3 (CASE_ID 4) done.
Row 4 (CASE_ID 5) done.
Row 5 (CASE_ID 6) done.
Row 6 (CASE_ID 7) done.
Row 7 (CASE_ID 8) done.
Row 8 (CASE_ID 9) done.
Row 9 (CASE_ID 10) done.
Row 10 (CASE_ID 11) done.
Row 11 (CASE_ID 12) done.
Row 12 (CASE_ID 13) done.
Row 13 (CASE_ID 14) done.
Row 14 (CASE_ID 15) done.
Row 15 (CASE_ID 16) done.
Row 16 (CASE_ID 17) done.
Row 17 (CASE_ID 18) done.
Row 18 (CASE_ID 19) done.
Row 19 (CASE_ID 20) done.
Row 20 (CASE_ID 21) done.
Row 21 (CASE_ID 22) done.
Row 22 (CASE_ID 23) done.
Row 23 (CASE_ID 24) done.
Row 24 (CASE_ID 25) done.
saved incident_reports_batch_1.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download started.


Note: There are 149 rows in the Transcript Data excel.

In [65]:
generate_incident_reports_batch(140)

Generating reports for rows 140–149 (10 total) — 2025-11-14 15:47:02
Row 140 (CASE_ID 141) done.
Row 141 (CASE_ID 142) done.
Row 142 (CASE_ID 143) done.
Row 143 (CASE_ID 144) done.
Row 144 (CASE_ID 145) done.
Row 145 (CASE_ID 146) done.
Row 146 (CASE_ID 147) done.
Row 147 (CASE_ID 148) done.
Row 148 (CASE_ID 149) done.
Row 149 (CASE_ID 150) done.
saved incident_reports_batch_6.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download started.


In [66]:
#preview output so we dont have to download every time
def preview_outputs(path_glob=f"{OUTPUT_PREFIX}*.xlsx", max_reports=1): #max only one report for now
    import glob
    files = sorted(glob.glob(path_glob))
    if not files:
        print("No batch files found yet. Run a batch first.")
        return
    latest = files[-1]
    print("Previewing file:", latest)
    df_out = pd.read_excel(latest)
    for i, row in df_out.head(max_reports).iterrows():
        print("\n" + "="*80)
        print(f"CASE_ID: {row.get('CASE_ID')}")
        print("-"*80)
        print(row.get("Incident_Report", ""))

preview_outputs()

Previewing file: incident_reports_batch_6.xlsx

CASE_ID: 141
--------------------------------------------------------------------------------
CASE_ID: 141

On November 21, 2025, at approximately 22:02 hours, I, Officer Ebner (#2339), responded to a report of a sexual assault at a local restaurant, specifically in the staff locker corridor. The weather was cloudy with a temperature of 69°F. Upon arrival, I made contact with the reporting party, identified as Camila, who stated that after the restaurant had closed, a male coworker touched her chest without consent. Camila reported that she immediately said “No” and left the area.

During the interview, I noted that the kitchen corridors in the establishment are typically crowded during operating hours, and brief physical contact can sometimes be incidental due to the confined space. I asked Camila if there was any physical restraint or injury resulting from the incident. She confirmed that there were no marks or injuries and that she did

# Bias Scale (Attempt 1)

In [ ]:
# three tier bias scale

import re, glob
import pandas as pd
from textblob import TextBlob

#configuring batch into glob
BATCH_GLOB = "incident_reports_batch_*.xlsx"

#Tier 1 (Neutral/Objective)
#Tier 2 (Subjective / victim-biasing)
#Tier 3 (Undermining MPC elements)
BATCH_TIER_MAP = {
    1: "Tier1_Neutral",
    2: "Tier2_Subjective",
    3: "Tier3_LegalUndermining",
}

COL_CASE = "CASE_ID"
COL_TEXT = "Incident_Report"


def load_batches(pattern=BATCH_GLOB):
    #load all batched files
    files = sorted(glob.glob(pattern))
    if not files:
        raise RuntimeError(f"No files found matching {pattern}. Run generation first.")
    frames = []
    for path in files:
        m = re.search(r"batch_(\d+)\.xlsx$", path)
        batch_num = int(m.group(1)) if m else None
        dfb = pd.read_excel(path)
        dfb["_batch_num"] = batch_num
        frames.append(dfb)
    out = pd.concat(frames, ignore_index=True)
    out["Tier"] = out["_batch_num"].map(BATCH_TIER_MAP).fillna("Unknown")
    return out

reports = load_batches()
print("Loaded rows:", len(reports))
reports.head(3)


Loaded rows: 25


,CASE_ID,Transcript,Incident_Report,_batch_num,Tier
0,1,[Body-Worn Camera Transcript — Officer: P.O. P...,"CASE_ID: 1\n\nOn April 6, 2025, at approximate...",1,Tier1_Neutral
1,2,[Body-Worn Camera Transcript — Officer: Sgt. R...,"CASE_ID: 2\n\nOn April 9, 2025, at approximate...",1,Tier1_Neutral
2,3,[Body-Worn Camera Transcript — Officer: P.O. M...,"CASE_ID: 3\n\nOn April 12, 2025, at approximat...",1,Tier1_Neutral


Code above sets three defined tiers as described in the "Linguistic Research" Google Doc. We start by defining column names: Case_ID and Incident_Reports defined as COL_CASE and COL_TEXT, respectively.

In [ ]:
#limited list of words, would need to expand
# Tier 1 — Neutral/Objective indicators
  #direct quotes
  #time/place details
  #action statements (e.g., "subject walked away", factual "stated they did not consent")
T1_QUOTE_RE = re.compile(r"[“”\"']", re.I)
T1_TIME_RE  = re.compile(r"\b(\d{1,2}:\d{2}\s?(?:am|pm)?)\b|\b(\d{1,2}\s?(?:am|pm))\b|\b(\d{4}-\d{2}-\d{2})\b", re.I)
T1_PLACE_WORDS = {
    "street","st.","st","avenue","ave","road","rd","boulevard","blvd","lane","ln",
    "drive","dr","park","city","county","block","intersection","apt","apartment","unit", "lakeside"
}
# neutral action patterns
T1_ACTION_PHRASES = {
    "subject walked away","victim stated they did not consent","victim stated no",
    "victim reported","officer observed","witness stated","was located at","arrived at"
}

# tier 2 — subjective language potentially biasing against victim
SKEPTICAL_QUALIFIERS = {
    "alleges","alleged","claims","claimed","supposedly","states she felt","states he felt",
    "she claims","he claims"
}
DEMEANOR_PHRASES = {
    "appeared unusually calm","did not cry","seemed detached","unemotional","spoke matter-of-fact"
}
VICTIM_CHARACTER = {
    "promiscuous","bad reputation","weak victim","sex worker","prostitute","regular drug user",
    "addict","mental"
}
INTOXICATION_FOCUS = {"intoxicated","heavily intoxicated","under the influence","inebriated","drinking"}
TIME_GAP_PHRASES = {"did not report until","occurred two nights ago","later said it was around"}
SELF_BLAME_PHRASES = {"i shouldn't have gone","i feel responsible","i am responsible","my fault"}
RELATIONSHIP_CONTEXT = {"prior relationship","knew each other"}

# tier 3 — language undermining MPC elements (force/threat, incapacitation, lack of consent)
EUPHEMISMS = {"intercourse occurred","sexual relations","had sex","the incident","sex happened"}
MINIMIZERS = {"no trauma","no fear"}
VAGUE_ALCOHOL = {"both parties had been drinking"}

# legal element coverage
FORCE_TERMS   = {"grabbed","pushed","hit","struck","choked","threatened","forced","held down","pinned","pushed her onto the bed","pushed him onto the bed"}
CONSENT_TERMS = {"without consent","said no","told him to stop","told her to stop","refused","non-consensual","rape","sexual assault","penetration without consent"}
INCAP_TERMS   = {"unconscious","slurred speech","impaired","vomiting","memory loss","blackout","found unconscious","could not stand","substantially impaired"}

# non-consensual explicit terms (contrast with euphemisms)
NONCONS_EXPLICIT = {"rape","sexual assault","penetration without consent","non-consensual"}


Categories that came directly from the "Linguistic Triggers" Google doc. Phrases are pretty limited; I would either add more phrases to each category to avoid overfitting to our specific transcripts or explore a way for the code use these phrases as a base and be able to identify other phrases as these same categories on its own.

In [ ]:
#extracting features

def _txt(x):
    return "" if pd.isna(x) else str(x)

#lowercase
def _low(x):
    return _txt(x).lower()

#counts how many times the target words appear as substrings in the transcript
def count_contains(text, terms):
    t = _low(text)
    return sum(1 for w in terms if w in t)

def has_regex(text, regex):
    return 1 if regex.search(_txt(text)) else 0

def count_place_words(text, place_words=T1_PLACE_WORDS):
    t = f" {_low(text)} "  # pad for simple word boundary
    return sum(t.count(f" {w} ") for w in place_words)

def count_action_phrases(text, phrases=T1_ACTION_PHRASES):
    t = _low(text)
    return sum(1 for p in phrases if p in t)

def sentiment_subjectivity(text):
    tb = TextBlob(_txt(text))
    return tb.sentiment.polarity, tb.sentiment.subjectivity

def extract_features(report_text: str) -> pd.Series:
    # Sentiment
    pol, sub = sentiment_subjectivity(report_text)

    # Tier 1 (objective) indicators
    t1_quotes      = has_regex(report_text, T1_QUOTE_RE)
    t1_time        = has_regex(report_text, T1_TIME_RE)
    t1_place_hits  = count_place_words(report_text)
    t1_actions     = count_action_phrases(report_text)

    # Tier 2 (subjective/victim-biasing) indicators
    t2_skeptic     = count_contains(report_text, SKEPTICAL_QUALIFIERS)
    t2_demeanor    = count_contains(report_text, DEMEANOR_PHRASES)
    t2_character   = count_contains(report_text, VICTIM_CHARACTER)
    t2_intox       = count_contains(report_text, INTOXICATION_FOCUS)
    t2_timegap     = count_contains(report_text, TIME_GAP_PHRASES)
    t2_selfblame   = count_contains(report_text, SELF_BLAME_PHRASES)
    t2_rel         = count_contains(report_text, RELATIONSHIP_CONTEXT)

    # Tier 3 (undermining MPC) indicators
    t3_euph        = count_contains(report_text, EUPHEMISMS)
    t3_min         = count_contains(report_text, MINIMIZERS)
    t3_vaguealc    = count_contains(report_text, VAGUE_ALCOHOL)

    # Legal element coverage (presence counts)
    legal_force    = count_contains(report_text, FORCE_TERMS)
    legal_consent  = count_contains(report_text, CONSENT_TERMS)
    legal_incap    = count_contains(report_text, INCAP_TERMS)
    legal_noncons  = count_contains(report_text, NONCONS_EXPLICIT)

    return pd.Series({
        "Polarity": pol,
        "Subjectivity": sub,

        # Tier 1 signals (higher = more objective style)
        "T1_Quotes": t1_quotes,
        "T1_TimeMention": t1_time,
        "T1_PlaceWordHits": t1_place_hits,
        "T1_ActionPhrases": t1_actions,

        # Tier 2 signals (subjective / victim-biasing)
        "T2_SkepticalQual": t2_skeptic,
        "T2_Demeanor": t2_demeanor,
        "T2_Character": t2_character,
        "T2_IntoxFocus": t2_intox,
        "T2_TimeGap": t2_timegap,
        "T2_SelfBlame": t2_selfblame,
        "T2_Relationship": t2_rel,

        # Tier 3 signals (undermining MPC elements)
        "T3_Euphemisms": t3_euph,
        "T3_Minimizers": t3_min,
        "T3_VagueAlcohol": t3_vaguealc,

        # Legal coverage (presence; *lower* may indicate omission/minimization)
        "Legal_Force": legal_force,
        "Legal_Consent": legal_consent,
        "Legal_Incapacitation": legal_incap,
        "Legal_NonConsExplicit": legal_noncons,
    })
if COL_TEXT not in reports.columns:
    raise KeyError(f"Column '{COL_TEXT}' not found in loaded batch files. Available: {list(reports.columns)}")

features = reports[COL_TEXT].apply(extract_features)
reports_feats = pd.concat([reports[[COL_CASE,"Tier",COL_TEXT,"_batch_num"]], features], axis=1)

print("Rows with features:", len(reports_feats))
reports_feats.head(3)

Rows with features: 25


,CASE_ID,Tier,Incident_Report,_batch_num,Polarity,Subjectivity,T1_Quotes,T1_TimeMention,T1_PlaceWordHits,T1_ActionPhrases,...,T2_TimeGap,T2_SelfBlame,T2_Relationship,T3_Euphemisms,T3_Minimizers,T3_VagueAlcohol,Legal_Force,Legal_Consent,Legal_Incapacitation,Legal_NonConsExplicit
0,1,Tier1_Neutral,"CASE_ID: 1\n\nOn April 6, 2025, at approximate...",1,0.044919,0.363432,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
1,2,Tier1_Neutral,"CASE_ID: 2\n\nOn April 9, 2025, at approximate...",1,0.074359,0.366484,1.0,1.0,9.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,2.0
2,3,Tier1_Neutral,"CASE_ID: 3\n\nOn April 12, 2025, at approximat...",1,0.063174,0.370867,1.0,1.0,6.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2.0,3.0,0.0,2.0


The above code counts the exact number of times our specific category phrases appear in the transcript. With this, you can see exactly how many occurences are in each T1, T2, T3 category. As you can see, even T1 can contain "T3 features." These are just frequencies of hard-coded words; we can rework this!

In [ ]:
#tier based

agg_cols = [
    "Polarity","Subjectivity",
    "T1_Quotes","T1_TimeMention","T1_PlaceWordHits","T1_ActionPhrases",
    "T2_SkepticalQual","T2_Demeanor","T2_Character","T2_IntoxFocus","T2_TimeGap","T2_SelfBlame","T2_Relationship",
    "T3_Euphemisms","T3_Minimizers","T3_VagueAlcohol",
    "Legal_Force","Legal_Consent","Legal_Incapacitation","Legal_NonConsExplicit",
]

summary = reports_feats.groupby("Tier")[agg_cols].mean().round(3).reset_index()
summary


,Tier,Polarity,Subjectivity,T1_Quotes,T1_TimeMention,T1_PlaceWordHits,T1_ActionPhrases,T2_SkepticalQual,T2_Demeanor,T2_Character,...,T2_TimeGap,T2_SelfBlame,T2_Relationship,T3_Euphemisms,T3_Minimizers,T3_VagueAlcohol,Legal_Force,Legal_Consent,Legal_Incapacitation,Legal_NonConsExplicit
0,Tier1_Neutral,0.078,0.392,1.0,1.0,2.08,0.12,0.4,0.0,0.2,...,0.0,0.0,0.0,0.96,0.0,0.0,0.6,1.72,0.0,1.4


Code above clears the Case ID, Incident Report, and batch numbers from the dataset. Subjectivity shows whether text is more factual or opinion based. 0 represents objective and 1 represents highly subjective text. Polarity shows if the text is positive, negative, or neutral. 0 is negative and 1 is positive.

### Links
https://www.geeksforgeeks.org/python/python-textblob-sentiment-method/

https://realpython.com/python-keras-text-classification/

https://huggingface.co/tasks/zero-shot-classification

# Bias Scale (Attempt 2)

For this second attempt, we are focusing on specific labeling and legal coverage. Trying to expand further than just subjective/objective or positive/negative. We will ask the model specific questions relating to

1. The victim (Stance labels)
    * Is this sentence objective, skeptical of the victim, minimizing their experience, etc.
2. The context of the report (Legal labels)
    * Does this sentence describe force, lack of consent, incapacitation, etc.

We will later tie the three tiers into our findings.


In [ ]:
#!pip install transformers torch sentencepiece nltk

In [67]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [68]:
#basic setup
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [69]:
#loading in our zero-shot classifier
classifier = pipeline(
    "zero-shot-classification",
    model = "facebook/bart-large-mnli"
)

Device set to use cpu


Zero shot classification essentially allows us to train a model on a set of example labels, and then have it classify unseens examples. This way, we don't have to hard code specific phrases.

In [70]:
#creating our stance and legal labels/descriptions
#multiple stance labels
STANCE_OBJECTIVE = [
    "Objective and factual description of events without interpretation or judgment",
    "Direct quoting of the victim or suspect without added commentary",
    "Factual description of injuries, evidence, time, and location",
]

STANCE_VICTIM_SUPPORTIVE = [
    "Language that clearly supports the victim and recognizes lack of consent",
]

STANCE_VICTIM_UNDERMINING = [
    "Language that implies victim culpability or risky behavior, such as intoxication, clothing, or prior relationship",
    "Language that alleges victim character flaws such as promiscuity, mental instability, drug use, or reputation",
    "Language that questions the victim’s credibility through descriptions of demeanor such as being calm, detached, or unemotional",
    "Language that emphasizes delays in reporting or inconsistencies in the victim’s timeline",
    "Language that uses skeptical qualifiers such as alleges, claims, supposedly, or quotation marks around victim statements",
    "Language that includes victim self-blame statements without contextualizing trauma responses",
    "Language in which the officer expresses subjective doubts about the victim’s credibility or calls the victim weak",
]

STANCE_LEGAL_OBSCURING = [
    "Language that obscures or minimizes the use of force or threats",
    "Language that obscures or downplays the victim’s incapacitation",
    "Language that obscures explicit lack of consent or ignores non-consent signals",
    "Language that uses euphemisms or neutral terms to describe non-consensual sexual acts",
]

#legal labels

LEGAL_GOOD_COVERAGE = [
    "Describes specific acts of force or threats used against the victim",
    "Describes the victim being incapacitated or substantially impaired",
    "Describes explicit lack of consent or resistance from the victim",
]

LEGAL_EUPHEMISM_OR_OMISSION = [
    "Uses euphemistic or vague language instead of naming non-consensual sexual acts",
    "Omits or minimizes legally relevant details showing force, threat, incapacitation, or non-consent",
]

STANCE_LABELS = (
    STANCE_OBJECTIVE
    + STANCE_VICTIM_SUPPORTIVE
    + STANCE_VICTIM_UNDERMINING
    + STANCE_LEGAL_OBSCURING
)

LEGAL_LABELS = (
    LEGAL_GOOD_COVERAGE
    + LEGAL_EUPHEMISM_OR_OMISSION
)

def _avg(scores_dict, keys):
    vals = [scores_dict[k] for k in keys if k in scores_dict]
    return sum(vals) / len(vals) if vals else 0.0


Here we are using zero shot classification on a single sentence. We'll analyze and score one sentence on the specific biases created above.

In [71]:
#trying zero shot classification on a single sentence
def classify_sentence_zero_shot(sentence: str):
    # stance / bias
    stance = classifier(
        sentence,
        candidate_labels=STANCE_LABELS,
        multi_label=True
    )
    stance_scores = dict(zip(stance["labels"], stance["scores"]))

    # legal coverage
    legal = classifier(
        sentence,
        candidate_labels=LEGAL_LABELS,
        multi_label=True
    )
    legal_scores = dict(zip(legal["labels"], legal["scores"]))

    return {
        "sentence": sentence,
        "stance_scores": stance_scores,
        "legal_scores": legal_scores,
    }


Since our narratives/reports are multiple sentences, we can either
1. We can split the narrative by sentences and analyze each
2. Analyze the whole narrative at once

We will try both and determine how each can be used.

In [72]:
def compute_bias_score_from_zero_shot(result):
    stance = result["stance_scores"]
    legal  = result["legal_scores"]

    #group means
    obj      = _avg(stance, STANCE_OBJECTIVE)
    support  = _avg(stance, STANCE_VICTIM_SUPPORTIVE)
    underm   = _avg(stance, STANCE_VICTIM_UNDERMINING)
    obsc_st  = _avg(stance, STANCE_LEGAL_OBSCURING)

    legal_good   = _avg(legal, LEGAL_GOOD_COVERAGE)
    legal_euph   = _avg(legal, LEGAL_EUPHEMISM_OR_OMISSION)

    #scoring logic
    #higher = more biased against victim / more legal obscuring
    raw = 0.0

    #victim-undermining narrative and legal-obscuring narrative
    raw += 2.0 * underm
    raw += 1.5 * obsc_st

    #euphemistic / omission on legal axis
    raw += 1.5 * legal_euph

    #subtract supportive / objective / good legal coverage
    raw -= 1.5 * support
    raw -= 0.5 * obj
    raw -= 0.5 * legal_good

    #clip to a nice range [0, 1] then to [0, 10] if you want a 0–10 scale
    #shift so that negative raw becomes 0
    shifted = max(raw, 0.0)
    #squish big numbers
    normalized = min(shifted, 3.0) / 3.0    # 0–1
    score_0_10 = normalized * 10.0          # 0–10

    return {
        "raw": raw,
        "normalized_0_1": normalized,
        "bias_score_0_10": score_0_10,
        "components": {
            "objective": obj,
            "victim_supportive": support,
            "victim_undermining": underm,
            "stance_legal_obscuring": obsc_st,
            "legal_good_coverage": legal_good,
            "legal_euphemism_or_omission": legal_euph,
        }
    }


Here we test on an example to see the output scores on the sentence. The code below analyzes a real sentence from our generated reports.

In [73]:
narr = """
Keira exhibited no visible physical injuries but reported experiencing anxiety and shaking.
"""
analysis = classify_sentence_zero_shot(narr)
final = compute_bias_score_from_zero_shot(analysis)
final["bias_score_0_10"], final["components"]

(7.456686731403885,
 {'objective': 0.08286955638322979,
  'victim_supportive': 0.785584032535553,
  'victim_undermining': 0.8098929694720677,
  'stance_legal_obscuring': 0.7058760523796082,
  'legal_good_coverage': 0.6246224840482076,
  'legal_euphemism_or_omission': 0.7270187139511108})

If we analyze an entire narrative sentence by sentence, the process would take a long time to complete for each narrative. Because of this, I think we could use sentence analysis is we want to look at a specific narrative in more detail. For example, after we determine which reports are "tier 3," we can analyze a specific tier 3 prompt and see what sentences scored the highest.

The narrative below was the result of a Tier 3 transcript (Case ID: 141). Next, we experiment with analyzing the whole narrative at once.

In [76]:
#real tier 3 narrative (pasting it word for word for testing purposes; we'll automate this to pull out the summary from the reports later)
narr = """
CASE_ID: 141

On November 21, 2025, at approximately 22:02 hours, I, Officer Ebner (#2339), responded to a report of a sexual assault at a local restaurant, specifically in the staff locker corridor. The weather was cloudy with a temperature of 69°F. Upon arrival, I made contact with the reporting party, identified as Camila, who stated that after the restaurant had closed, a male coworker touched her chest without consent. Camila reported that she immediately said “No” and left the area.

During the interview, I noted that the kitchen corridors in the establishment are typically crowded during operating hours, and brief physical contact can sometimes be incidental due to the confined space. I asked Camila if there was any physical restraint or injury resulting from the incident. She confirmed that there were no marks or injuries and that she did not shout or call out during the encounter.

I further inquired about the context and whether there might have been any misunderstanding or lack of awareness on the part of the alleged offender. Camila mentioned that coworkers often tease each other, suggesting the possibility that the male employee may not have been fully aware of the inappropriateness of his actions.

There was no CCTV footage available from the corridor where the incident occurred, nor were there any witnesses present to corroborate the report. Camila did not have any photographic evidence, and no Sexual Assault Nurse Examiner (SANE) or other medical resources were utilized or requested.

Based on the information gathered, there was no indication of compulsion, physical injury, or incapacitation. The intent behind the contact was ambiguous, and the incident appeared to be more of an administrative concern rather than a criminal matter. I advised Camila of her options and the procedures for reporting any future incidents. The case was documented accordingly.

At 22:55 hours, I cleared the scene with no further action taken at that time. Officer Fox (#2577) was present as secondary during the response and assisted as needed.

Report completed by Officer Ebner (#2339).
"""
analysis = classify_sentence_zero_shot(narr)
final = compute_bias_score_from_zero_shot(analysis)
final["bias_score_0_10"], final["components"]

(5.923496468199624,
 {'objective': 0.8039783239364624,
  'victim_supportive': 0.9334840774536133,
  'victim_undermining': 0.7599323391914368,
  'stance_legal_obscuring': 0.8152101635932922,
  'legal_good_coverage': 0.8126507798830668,
  'legal_euphemism_or_omission': 0.8286064565181732})

In [77]:
#real tier 3 narrative (pasting it word for word for testing purposes; we'll automate this to pull out the summary from the reports later)

narr =""" CASE_ID: 142

On November 22, 2025, at approximately 21:04 hours, I, Officer Doyle (#3198), along with Officer Kravitz (#2902), responded to a call regarding a reported sexual assault at the 700 Grand Hotel, Room 605. The weather was cloudy with a temperature of 70°F. Upon arrival, we made contact with the complainant, identified as Lena, who provided an account of the incident.

Lena stated that after consuming drinks at the hotel bar, an individual placed his hand inside her dress without her consent. She clearly told him "No" and indicated her desire for him to leave by pointing toward the door. According to her statement, the individual attempted the same action once more despite her refusal.

During the initial assessment, I considered the context of the interaction, noting that the complainant and the subject had been flirting at the bar prior to the incident. I explained that while flirtation can suggest mutual interest, a single verbal refusal, such as "No," might sometimes be interpreted as ambiguous or mixed messaging in social situations. However, I emphasized that for an incident to meet the threshold of sexual assault under the law, there must be evidence of force, restraint, or injury.

I inquired about any physical injuries or signs of struggle, but Lena reported none. There were no visible injuries or indications of physical coercion. Additionally, no medical examination by a Sexual Assault Nurse Examiner (SANE) was conducted or requested at the time of our investigation. I also noted that keycard logs, hallway surveillance footage, and potential witnesses were not reviewed or considered during the initial report.

Based on the information gathered, including the absence of force, restraint, injury, or corroborating evidence, I concluded that the incident did not meet the prosecutorial standards for sexual assault. The complainant’s description suggested unwanted contact, but without additional elements such as physical coercion or injury, the case was classified as non-prosecutable.

Throughout the interaction, I maintained a professional and empathetic demeanor, ensuring Lena was informed of her options and resources, although no immediate referrals to support services were made. The entire response and investigation at the scene lasted until approximately 21:57 hours, at which point we cleared the location.

In summary, the incident involved unwanted physical contact following a social interaction characterized by mutual flirtation. The complainant’s refusal was noted, but the lack of force, injury, or corroborative evidence led to the determination that the case did not meet the criteria for further legal action.

Report completed by Officer Doyle (#3198).
"""

analysis = classify_sentence_zero_shot(narr)
final = compute_bias_score_from_zero_shot(analysis)
final["bias_score_0_10"], final["components"]


(7.230329604612456,
 {'objective': 0.9440373380978903,
  'victim_supportive': 0.992122232913971,
  'victim_undermining': 0.905626118183136,
  'stance_legal_obscuring': 0.9387419372797012,
  'legal_good_coverage': 0.9450315237045288,
  'legal_euphemism_or_omission': 0.9216343462467194})

In [78]:
#real tier 3 narrative (pasting it word for word for testing purposes; we'll automate this to pull out the summary from the reports later)

narr =""" CASE_ID: 143

On November 22, 2025, at approximately 07:06 hours, I, Officer Li (#2631), along with Officer Gomez (#2740), responded to a call regarding a sexual assault report at the Riverwalk area, specifically under the bridge at mile marker 1.8. The weather was breezy with a temperature of 75°F. Our time on scene was from 07:06:58 to 08:00:21.

Upon arrival, I made contact with the reporting party, who identified herself as June. June stated that while she was near the railing, an unidentified male touched her in a manner she found inappropriate. She clearly recalled saying, “No, stop,” and then stepping away from the individual. June’s account suggested that the contact was unwanted, but she did not indicate any physical restraint or injury resulting from the encounter.

During the interview, I explained the legal standards relevant to her report. I informed June that unless the suspect had restrained her or caused physical harm, the incident might not meet the statutory requirements for a sexual assault under current law. I further clarified that the element of mens rea—meaning the suspect’s knowledge or intent to commit a crime—was unsettled in this case, especially since the suspect did not acknowledge her verbal refusal.

No physical evidence was collected at the scene, and June declined or was not offered a Sexual Assault Nurse Examiner (SANE) examination or other medical resources. Given the absence of compulsion, injury, or clear acknowledgment of non-consent by the suspect, I summarized the situation as ambiguous with respect to criminal intent and compulsion.

Throughout the investigation, I maintained a professional and empathetic demeanor, ensuring June felt heard and supported despite the limitations of the case. I advised her of her options should she wish to pursue further action or require additional support services.

After completing the initial report and ensuring the scene was secure, we cleared the location at 08:00:21. The case remains documented under CAD# 25-242200, with no further immediate investigative steps taken due to the lack of evidence and statutory criteria.

This concludes my report on the incident at Riverwalk under the bridge on November 22, 2025.
"""

analysis = classify_sentence_zero_shot(narr)
final = compute_bias_score_from_zero_shot(analysis)
final["bias_score_0_10"], final["components"]


(5.8728946769048305,
 {'objective': 0.7227609157562256,
  'victim_supportive': 0.9650546312332153,
  'victim_undermining': 0.7686058708599636,
  'stance_legal_obscuring': 0.8270125985145569,
  'legal_good_coverage': 0.8913682897885641,
  'legal_euphemism_or_omission': 0.8258562088012695})

Now experimenting with tier 1 narratives.

In [84]:
narr = """CASE_ID: 1

On April 6, 2025, at approximately 02:06 hours, I, Police Officer Patel (#5123), along with Secondary Officer Nguyen (#4419), responded to a reported sexual assault at the rear parking lot of 1190 N. Cedar Ave., City of Foxridge. The weather was light rain with a temperature of 47°F. Upon arrival, we observed a gray four-door sedan parked with the rear passenger door slightly ajar. A female adult, later identified as Ari [last name withheld], was seated on the curb near the vehicle, wrapped in a blanket provided by her friend, Kira [last name withheld], who stood nearby.

I introduced myself and informed Ari that we were recording on our body cameras and asked for her consent to speak on scene, which she granted. Ari appeared alert and oriented, with steady breathing and no visible external injuries on her face or forearms. She reported soreness in her inner thighs. Kira confirmed she had called 911 at 02:01 after receiving a distressed phone call from Ari at 01:58.

Ari stated that the suspect, identified as Luis, a coworker, had put her in the backseat of the vehicle against her will. She described how Luis had offered to walk her to her car after leaving a bar around 01:35. Upon arrival at the parking lot, Luis asked to sit in the car, but Ari expressed her desire to leave. Luis then opened the back door and nudged her inside despite her repeated verbal refusals, saying “stop” and “no.” Ari attempted to push him away, and she said, “He held my wrists.” She described Luis as a Hispanic male in his mid-30s, approximately 5’10” with a medium build, wearing a navy jacket, jeans, and a black cap. Luis was last seen leaving the lot in a dark SUV around 01:55.

We requested EMS for a medical evaluation and a Sexual Assault Nurse Examiner (SANE) exam at the hospital. Ari consented to officers photographing the interior of the vehicle prior to EMS arrival. Upon inspection, the gray sedan’s back seat fabric was torn approximately four inches at the center seam, and tissue paper was found on the floorboard. A condom wrapper was located near the rear passenger footwell. The vehicle’s license plate was partially obscured by mud, and a temporary paper permit was visible inside the rear window. The permit was later confirmed to be tied to dealership inventory and not registered to Ari.

I canvassed the area and spoke with the night attendant at a nearby laundromat, who agreed to request CCTV footage covering the time frame from 01:30 to 02:10. The footage could potentially capture Luis’s departure in the dark SUV.

At 02:11, EMS arrived, and the lead paramedic, Renee, explained the medical and forensic options to Ari, who elected to proceed with the SANE exam. Ari consented to leave the vehicle in place and to the release of the sexual assault kit to law enforcement after the exam.

At the hospital, the sexual assault kit and clothing items (black leggings, gray hoodie, and underwear) were sealed and received by Officer Nguyen at 02:47. The evidence was logged into the CAD and evidence module accordingly. An advocate arrived at 02:58 to provide support to Ari.

We cleared the hospital at 03:03, concluding our involvement. The victim’s identity was verified, direct quotes were documented, and photographic evidence of the vehicle interior was collected with consent. No physical evidence was collected on scene due to weather conditions and pending the SANE exam. Follow-up actions include reviewing CCTV footage and continuing the investigation into the suspect Luis.

End of report.
"""

analysis = classify_sentence_zero_shot(narr)
final = compute_bias_score_from_zero_shot(analysis)
final["bias_score_0_10"], final["components"]

(3.0233087857800816,
 {'objective': 0.6536994179089864,
  'victim_supportive': 0.98317551612854,
  'victim_undermining': 0.6534506593431745,
  'stance_legal_obscuring': 0.7589218616485596,
  'legal_good_coverage': 0.7661318182945251,
  'legal_euphemism_or_omission': 0.4309249445796013})

Now, we will incorporate our original tiers. Bias level?